In [1]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [2]:
llm = Ollama(model="llama2")
embeddings_provider = OllamaEmbeddings()

In [3]:
loader = WebBaseLoader(
    ["https://en.wikipedia.org/wiki/2024_Summer_Olympics"
     ]
)

docs = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter()
document_chunks = text_splitter.split_documents(docs)

In [5]:
vector_store = DocArrayInMemorySearch.from_documents(document_chunks, embeddings_provider)

In [6]:
template = """Answer the question based only on the following context:
                {context}

                Question: {question}
                """
prompt = ChatPromptTemplate.from_template(template)

In [7]:
retriever = vector_store.as_retriever(
                search_type="similarity"
            )
output_parser = StrOutputParser()

In [8]:
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | llm | output_parser


In [9]:
result = chain.invoke("Where are the 2024 summer olympics being held?")
print(result)

Answer: The 2024 Summer Olympics are being held in Paris, France.


In [10]:
result = chain.invoke("What are the new sports that are being added for the 2024 summer olympics?")
print(result)

The 2024 Summer Olympics will feature several new sports and events, including:

1. Breakdancing: This sport has been proposed as a new Olympic sport for the 2024 Games, and is currently under consideration by the International Olympic Committee (IOC).
2. Skateboarding: Skateboarding was introduced as an Olympic sport in 2020, and will be featured again in 2024.
3. Surfing: Surfing was also introduced as an Olympic sport in 2020, and will continue to be featured in 2024.
4. Sport Climbing: This sport has been added to the Olympic program for the first time in 2020, and will continue to be featured in 2024.
5. Karate: Karate was approved as an Olympic sport in 2018, and will make its debut at the 2024 Games.
6. Breaking (Wushu): This sport has been proposed as a new Olympic sport for the 2024 Games, but is still under consideration by the IOC.
7. Chess: Chess was considered as an Olympic sport in 2019, but it was ultimately decided that it would not be included in the 2024 Olympics.
8. 